In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANG_SMITH")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
# TAVILY 타빌리 LLM 문서 리트리브 용 검색기

from langchain_community.tools.tavily_search import TavilySearchResults

In [22]:
# 도구들 불러오기

search = TavilySearchResults(max_results=1)
search_results = search.invoke("what is the weather in SF")

In [23]:
tools = [search]

In [24]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

# model = ChatAnthropic(model="claude-3-sonnet-20240229")
model = ChatOpenAI(model="gpt-3.5-turbo")

In [25]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I help you today?'

In [26]:
# 모델에 도구 바인딩 하는 작업

model_with_tools = model.bind_tools(tools)

In [27]:
# 일반 메시지 tool이 필요가 없음

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [28]:
# tool의 사용이 필요한 메시지로

response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_CfUYv0GbkHgvLy35spBkcDLM', 'type': 'tool_call'}]


In [65]:
# langgraph에서 제공하는 agent로 사용

from langgraph.prebuilt import create_react_agent

# agent_executor = create_react_agent(model, tools, debug=True)
agent_executor = create_react_agent(model, tools)

In [56]:
# 일반 메시지

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='hi!')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='hi!')]
[1:tasks] Starting step 1 with 1 task:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='hi!', id='467c46a4-f74b-4637-bc3c-9df99ef8b22d')]}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 91, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-55baa40c-22d0-4335-a5bf-a0b674d691ef-0', usage_metadata={'input_tokens': 91, 'output_tokens': 10, 'total_tokens': 101})]


[HumanMessage(content='hi!', id='467c46a4-f74b-4637-bc3c-9df99ef8b22d'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 91, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-55baa40c-22d0-4335-a5bf-a0b674d691ef-0', usage_metadata={'input_tokens': 91, 'output_tokens': 10, 'total_tokens': 101})]

In [66]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the president of korea?")]}
)
response["messages"]

GraphRecursionError: Recursion limit of 25 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

In [51]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='whats the weather in sf?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='whats the weather in sf?')]
[1:tasks] Starting step 1 with 1 task:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='whats the weather in sf?', id='7430ce81-320a-44ff-886f-a606d30d4e41')]}
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fnoHQMdvKgIpfbBVCz5zf9jT', 'function': {'arguments': '{"config":{"tags":["weather","San Francisco"],"run_name":"Weather in SF"},"args":["weather in San Francisco"]}', 'name': 'Search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 96, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db7e67f1-c061-43e0-b367-f776838d4602-0', tool_calls=[{

- 에러 발생 진행 x
- https://github.com/langchain-ai/langgraph/issues/1069